In [39]:
import plotly.express as px
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

In [40]:
from itertools import chain

res_dir = os.path.join('results', 
                       'how_long_heuristic_is_better')

ns = list(range(1, 11 + 1))
families = ['euclidean', 'noise', 'truncated_urn', 'resampling']

res_dirs_wout_trials = {
    (n, family): os.path.join(res_dir, f'30x60_{n}', 'basin_hopping', family) for family in families for n in ns}


res_dirs_w_trials = {(n, family): list(sorted(name for name in os.listdir(res_dir_wout_trials) if os.path.isdir(os.path.join(res_dir_wout_trials, name)))) for (n, family), res_dir_wout_trials in res_dirs_wout_trials.items()}


In [41]:
res = []
drop_tail = False
for (n, family), trial_files in res_dirs_w_trials.items():
    if drop_tail:
        trial_files = trial_files[:-1]
    for trial_file in trial_files:
        with open(os.path.join(res_dirs_wout_trials[(n, family)], trial_file, 'new_heuristics_approvalwise_vectors.txt')) as f:
            lines = f.readlines()
            count = len(lines) - 1
            i_trial = int(trial_file[-1])
            res.append({'n': n, 'family': family, 'trial': i_trial, 'count': count})

df = pd.DataFrame(res)
df = df.groupby(['family', 'n']).agg({'count': ['mean', 'std']})

df['count']['mean'] -= df.index.get_level_values(1)
df

count          
                       mean       std
family        n                      
euclidean     1    0.000000  0.000000
              2    0.000000  0.000000
              3   -1.000000  0.000000
              4    0.000000  0.000000
              5    1.600000  3.435113
              6    4.000000  6.055301
              7    2.500000  4.358899
              8    4.666667  4.932883
              9    5.000000  7.071068
              10   5.000000       NaN
              11  -0.500000  3.511885
noise         1    1.666667  2.081666
              2    1.000000  1.000000
              3    1.500000  2.121320
              4    3.000000  4.242641
              5    1.000000       NaN
              6    1.000000       NaN
              7    1.000000       NaN
              8   -1.000000       NaN
              9   -2.000000       NaN
              10  -3.000000       NaN
              11  -3.000000       NaN
resampling    1    0.000000  0.000000
              2    0.250000  1.500000
              3   -0.600000  0.547723
              4    0.000000       NaN
              5    4.000000       NaN
              6    3.000000       NaN
              7    4.000000       NaN
              8    0.000000       NaN
              9   -1.000000       NaN
              10  -2.000000       NaN
              11  14.000000       NaN
truncated_urn 1    0.000000  0.000000
              2    8.666667  0.577350
              3    9.000000       NaN
              4   12.000000  1.414214
              5   10.000000       NaN
              6    7.000000       NaN
              7    8.000000       NaN
              8    7.000000       NaN
              9    6.000000       NaN
              10  12.000000       NaN
              11  31.000000       NaN

In [42]:
px.line(df['count'].reset_index(), x='n', y='mean', color='family', error_y='std')